In [1]:
import os
import random
import numpy as np
import torch
import re

from datasets import load_dataset

/opt/homebrew/Caskroom/miniforge/base/envs/quantum2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dev = (
    torch.device("mps")
    if torch.backends.mps.is_available()
    else torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
)

print(dev)

seed = 42

os.environ["PYTHONHASHSEED"] = str(seed)
random.seed(seed)
np.random.seed(seed)

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

mps


In [3]:
cache_dir = "./.cache/datasets"

imdb = load_dataset("imdb", cache_dir=cache_dir)
yelp = load_dataset("yelp_polarity", cache_dir=cache_dir)
amazon = load_dataset("amazon_polarity", cache_dir=cache_dir)

In [4]:
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [5]:
yelp

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 560000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 38000
    })
})

In [6]:
amazon

DatasetDict({
    train: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 3600000
    })
    test: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 400000
    })
})

In [7]:
imdb_train_val = amazon["train"].train_test_split(test_size=0.2, seed=42)  # type: ignore
imdb_train_dataset = imdb_train_val["train"]
imdb_val_dataset = imdb_train_val["test"]

In [ ]:
print([len([label for label in imdb_train_dataset["label"] if label == 1])])

In [8]:
imdb_train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 20000
})

In [9]:
imdb_train_dataset["text"]

['Stage adaptations often have a major fault. They often come out looking like a film camera was simply placed on the stage (Such as "Night Mother"). Sidney Lumet\'s direction keeps the film alive, which is especially difficult since the picture offered him no real challenge. Still, it\'s nice to look at for what it is. The chemistry between Michael Caine and Christopher Reeve is quite brilliant. The dynamics of their relationship are surprising. Caine is fantastic as always, and Reeve gets one of his few chances to really act.<br /><br />I confess that I\'ve never seen Ira Levin\'s play, but I hear that Jay Presson Allen\'s adaptation is faithful. The script is incredibly convoluted, and keeps you guessing. "Deathtrap" is an enormously entertaining film, and is recommended for nearly all fans of stage and screen.<br /><br />7.4 out of 10',
 "'The Rookie' was a wonderful movie about the second chances life holds for us and also puts an emotional thought over the audience, making them r

In [10]:
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r"<.*?>", "", text)
    # Replace single quotes that are not preceded by a backslash
    text = re.sub(r"(?<!\\)'", '"', text)
    return text

In [11]:
imdb_train_labels = imdb_train_dataset["label"]
imdb_val_labels = imdb_val_dataset["label"]

imdb_train_text = [
    clean_text(text) for _, text in enumerate(imdb_train_dataset["text"])
]
imdb_val_text = [clean_text(text) for _, text in enumerate(imdb_val_dataset["text"])]

In [12]:
from transformers import BertTokenizer, BertModel

model_name = "bert-base-uncased"
cache_folder = "./.cache/huggingface"
bert_tokenizer = BertTokenizer.from_pretrained(
    model_name,
    cache_dir=cache_folder,
    device_map="auto",
)
bert_model = BertModel.from_pretrained(
    model_name,
    cache_dir=cache_folder,
    device_map="auto",
)

/opt/homebrew/Caskroom/miniforge/base/envs/quantum2/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
# Print the special tokens
print("CLS token:", bert_tokenizer.cls_token)
print("SEP token:", bert_tokenizer.sep_token)
print("PAD token:", bert_tokenizer.pad_token)
print("UNK token:", bert_tokenizer.unk_token)
print("MASK token:", bert_tokenizer.mask_token)

# Alternatively, print all special tokens and their IDs
print("All special tokens:", bert_tokenizer.special_tokens_map)
print("All special tokens with IDs:", bert_tokenizer.special_tokens_map_extended)

CLS token: [CLS]
SEP token: [SEP]
PAD token: [PAD]
UNK token: [UNK]
MASK token: [MASK]
All special tokens: {'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}
All special tokens with IDs: {'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}


In [14]:
tokens = bert_tokenizer(
    imdb_train_text, return_tensors="pt", padding=True, truncation=True, max_length=128
).to(dev)

In [15]:
from pprint import pprint

pprint(tokens)

{'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]),
 'input_ids': tensor([[  101,  2754, 17241,  ...,  2963,  2008,   102],
        [  101,  1000,  1996,  ...,     0,     0,     0],
        [  101,  7929,  1010,  ...,  1012,  1045,   102],
        ...,
        [  101,  3087,  2040,  ...,  4025,  2000,   102],
        [  101,  2054,  2079,  ...,  2987,  1000,   102],
        [  101,  1045,  3342,  ...,  2876,  1000,   102]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])}


In [16]:
bert_tokenizer.convert_ids_to_tokens(tokens["input_ids"][0])

['[CLS]',
 'stage',
 'adaptations',
 'often',
 'have',
 'a',
 'major',
 'fault',
 '.',
 'they',
 'often',
 'come',
 'out',
 'looking',
 'like',
 'a',
 'film',
 'camera',
 'was',
 'simply',
 'placed',
 'on',
 'the',
 'stage',
 '(',
 'such',
 'as',
 '"',
 'night',
 'mother',
 '"',
 ')',
 '.',
 'sidney',
 'lu',
 '##met',
 '"',
 's',
 'direction',
 'keeps',
 'the',
 'film',
 'alive',
 ',',
 'which',
 'is',
 'especially',
 'difficult',
 'since',
 'the',
 'picture',
 'offered',
 'him',
 'no',
 'real',
 'challenge',
 '.',
 'still',
 ',',
 'it',
 '"',
 's',
 'nice',
 'to',
 'look',
 'at',
 'for',
 'what',
 'it',
 'is',
 '.',
 'the',
 'chemistry',
 'between',
 'michael',
 'caine',
 'and',
 'christopher',
 'reeve',
 'is',
 'quite',
 'brilliant',
 '.',
 'the',
 'dynamics',
 'of',
 'their',
 'relationship',
 'are',
 'surprising',
 '.',
 'caine',
 'is',
 'fantastic',
 'as',
 'always',
 ',',
 'and',
 'reeve',
 'gets',
 'one',
 'of',
 'his',
 'few',
 'chances',
 'to',
 'really',
 'act',
 '.',
 'i',
 

In [17]:
# with torch.no_grad():
#     # outputs = bert_model(**tokens)
#     outputs = bert_model(tokens["input_ids"], attention_mask=tokens["attention_mask"])
#     word_embeddings = outputs.last_hidden_state

In [ ]:
batch_size = 1024
input_ids = tokens["input_ids"]
attention_mask = tokens["attention_mask"]

all_word_embeddings = []

with torch.no_grad():
    for i in range(0, input_ids.size(0), batch_size):
        batch_input_ids = input_ids[i : i + batch_size]
        batch_attention_mask = attention_mask[i : i + batch_size]

        outputs = bert_model(batch_input_ids, attention_mask=batch_attention_mask)
        batch_word_embeddings = outputs.last_hidden_state

        all_word_embeddings.append(batch_word_embeddings)

word_embeddings = torch.cat(all_word_embeddings, dim=0)

In [ ]:
word_embeddings.shape

torch.Size([1, 128, 768])

In [ ]:
torch.save(word_embeddings, "imdb_word_embeddings.pt")

In [ ]:
word_embeddings = torch.load("imdb_word_embeddings.pt")

In [ ]:
word_embeddings.shape

128


In [ ]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components=8)
# X_train_pca = pca.fit_transform(word_embeddings.cpu().numpy())

ValueError: Found array with dim 3. PCA expected <= 2.

In [ ]:
# X_train_pca.shape

In [ ]:
import math
import torch
import torch.nn as nn
from torch import Tensor, device


class PositionalEncoder(nn.Module):

    def __init__(self, d_model: int, dropout=0.1, max_len=512, device=device("cpu")):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)
        )
        pe = torch.zeros(max_len, 1, d_model, device=device)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        seq_len = x.size(1)
        x = x + self.pe[: x.size(0)]
        return self.dropout(x)

In [ ]:
# train_data = [(int(label), text) for label, text in train_data]
# test_data = [(int(label), text) for label, text in test_data]

# print("train pos: ", len([label for label, text in train_data if label == 0]))
# print("train neg: ", len([label for label, text in train_data if label == 1]))
# print("test pos: ", len([label for label, text in test_data if label == 0]))
# print("test neg: ", len([label for label, text in test_data if label == 1]))

In [ ]:
# train_data[0:5]

In [ ]:
# type(train_data[0][0])

In [ ]:
# [float(label) for label in imdb_train_labels]